In [1]:
from topic_modelling.pipeline import run_topic_modelling_pipeline

In [2]:
run_topic_modelling_pipeline()

ERROR in run_topic_modelling_pipeline [Errno 2] No such file or directory: 'output_topics_data/topics_by_count.json'
Topic modelling on 10 survey responses
Extracting embedding from given topic 0 / 10


/home/sarthak/myenv/feedback-response-gen/src/utils/llm.py:80: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceBgeEmbeddings(
/home/sarthak/myenv/feedback-response-gen/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting embedding from given topic 1 / 10
Extracting topics from response 0 / 10
Reason for given NPS: Never had any issue as and usually get good service
Sending request to LLM
Received response from LLM.
llm response {
    "detected_topics" : [],
    "suggested_topics" : [],
    "escalation_topics" : []
}


----------------------------------------------------------


Extracting topics from response 1 / 10
Improvement needed: Sometimes hard to understand an operator when talking to them found it easier going into a store

Sending request to LLM
Received response from LLM.
llm response {
    "detected_topics" : ["customer_wants_cheaper_pricing_plans"],
    "suggested_topics" : ["poor_understandability_of_call_agents", "preference_for_in-store_interactions"],
    "escalation_topics": []
}


----------------------------------------------------------


Extracting topics from response 2 / 10
Improvement needed: I live semi-rural and cellphone coverage is really patchy. When we have powe